#### Отчиистка таблицы и обновление счётчика 

In [ ]:
TRUNCATE TABLE table_name

In [ ]:
-- Если таблица создаётся с полем id SERIAL PRIMARY KEY, то этой командой можно обновить её счётчик
TRUNCATE TABLE table_name RESTART IDENTITY;

#### Настройка ролей в базе данных

In [ ]:
select
    rolname,
    rolconfig
from pg_roles
where
    rolname in (
    'anon',
    'authenticated',
    'postgres',
    'service_role'
    -- ,<ANY CUSTOM ROLES>
    );

alter role postgres set statement_timeout = '10min'

NOTIFY pgrst, 'reload config';

#### Получить списком колонки таблицы

In [ ]:
SELECT string_agg(column_name, ', ') AS columns_list
FROM information_schema.columns
WHERE table_name = 'participants';

#### Создание представления по домашним результатам

In [ ]:
create view home_events as
    SELECT run, date, link, finishers, volunteers, avg_time, best_female_time, best_male_time
    FROM events
    WHERE run LIKE 'Петергоф Александрийский%'

In [ ]:
create view home_participants as
	SELECT run, date, link, finishers, volunteers, avg_time, best_female_time, best_male_time, position, name, profile_link, participant_id, finishes, clubs, age_group, age_grade, time, achievements
	FROM participants
	WHERE run LIKE 'Петергоф Александрийский%'

In [ ]:
create view home_volunteers as
    SELECT run, date, link, finishers, volunteers, avg_time, best_female_time, best_male_time, name, profile_link, participant_id, finishes, clubs, first_volunteer_info, volunteer_role
    FROM volunteers
    WHERE run LIKE 'Петергоф Александрийский%'

In [ ]:
DROP VIEW IF EXISTS view_name;
DROP VIEW view_name CASCADE;

#### Расчёт петергофских участников

In [ ]:
SELECT 
    p.name,
    MAX(LEFT(p.age_group, 1)) AS sex,
    p.profile_link,
    p.participant_id,
    MIN(TO_TIMESTAMP(p.time, 'HH24:MI:SS')::TIME) AS best_time,
    COUNT(*) AS n_finishes,
    SUM(CASE WHEN p.run LIKE 'Петергоф Александрийский%' THEN 1 ELSE 0 END) AS n_finishes_home,
    latest.max_date AS r_latest_date,
    latest.r_clubs
FROM 
    participants p
LEFT JOIN (
    SELECT 
        participant_id, 
        MAX(TO_DATE(date, 'DD.MM.YYYY')) AS max_date,  
        clubs as r_clubs
    FROM participants
    GROUP BY participant_id, r_clubs  
) latest ON p.participant_id = latest.participant_id
GROUP BY 
    p.name, p.profile_link, p.participant_id, latest.max_date, latest.r_clubs
HAVING
    SUM(CASE WHEN p.run LIKE 'Петергоф Александрийский%' THEN 1 ELSE 0 END) > 0
    AND p.profile_link != '';

#### Расчёт петергофских волонтёров

In [ ]:
SELECT 
    v.name,
    v.profile_link,
    v.participant_id,
    COUNT(DISTINCT v.date) AS n_volunteers,
    COUNT(DISTINCT CASE 
        WHEN v.run LIKE 'Петергоф Александрийский%' THEN v.date 
        ELSE NULL 
    END) AS n_volunteers_home,
    latest.max_date AS v_latest_date,
    latest.v_clubs
FROM 
    volunteers v
LEFT JOIN (
    SELECT 
        participant_id, 
        MAX(TO_DATE(date, 'DD.MM.YYYY')) AS max_date,  
        clubs AS v_clubs 
    FROM volunteers
    GROUP BY participant_id, clubs  
) latest ON v.participant_id = latest.participant_id
GROUP BY 
    v.name, v.profile_link, v.participant_id, latest.max_date, latest.v_clubs
HAVING
    SUM(CASE WHEN v.run LIKE 'Петергоф Александрийский%' THEN 1 ELSE 0 END) > 0
    AND v.profile_link != '';

#### Общая таблица по Петергофу

In [ ]:
create view petergof_summary as
SELECT 
    COALESCE(p.participant_id, v.participant_id) AS participant_id,
    COALESCE(p.name, v.name) AS name,
    p.sex,
    p.best_time,
    p.n_finishes,
    p.n_finishes_home,
    p.r_latest_date,
    v.n_volunteers,
    v.n_volunteers_home,
    v.v_latest_date,
    COALESCE(p.r_clubs, v.v_clubs) AS clubs,
    COALESCE(p.profile_link, v.profile_link) AS profile_link
FROM 
    home_participants p
FULL OUTER JOIN 
    home_volunteers v 
ON 
    p.participant_id = v.participant_id;

## Таблицы по рекордсменам, новичкам и вступившим в клубы 10/25/50/100

#### Рекорды

In [ ]:
create view view_records as
SELECT 
    profile_link,
    participant_id,
    name,
    time,
    position
FROM participants
WHERE 
achievements LIKE '%Личный рекорд!%' 
AND run LIKE 'Петергоф Александрийский%'
AND TO_DATE(date, 'DD.MM.YYYY') = (
    SELECT MAX(TO_DATE(date, 'DD.MM.YYYY')) AS date
    FROM participants
    )

#### Первый финиш на 5 верст

In [ ]:
create view view_first_finish as
SELECT 
    profile_link,
    participant_id,
    name,
    time,
    position
FROM participants
WHERE 
achievements LIKE '%Первый финиш на 5 вёрст%'
AND run LIKE 'Петергоф Александрийский%'
AND TO_DATE(date, 'DD.MM.YYYY') = (
    SELECT MAX(TO_DATE(date, 'DD.MM.YYYY')) AS date
    FROM participants
    )

#### Первый финиш в Петергофе

In [ ]:
create view view_first_finish_petergof as
SELECT 
    profile_link,
    participant_id,
    name,
    time, 
    position,
    n_finishes
FROM participants
WHERE 
achievements LIKE '%Первый финиш на Петергоф Александрийский%'
AND run LIKE 'Петергоф Александрийский%'
AND TO_DATE(date, 'DD.MM.YYYY') = (
    SELECT MAX(TO_DATE(date, 'DD.MM.YYYY')) AS date
    FROM participants
    )

#### Первое волонтерство на 5 верст

In [ ]:
create view view_first_volunteer as
SELECT 
profile_link,
participant_id,
name
FROM volunteers
WHERE first_volunteer like '%Первое волонтёрство на 5 вёрст%'
AND run LIKE 'Петергоф Александрийский%'
AND TO_DATE(date, 'DD.MM.YYYY') = (
    SELECT MAX(TO_DATE(date, 'DD.MM.YYYY')) AS date
    FROM volunteers
    )

#### Первое волонтерство в Петергофе

In [ ]:
create view view_first_volunteer_petergof as
SELECT 
profile_link,
participant_id,
name
FROM volunteers
WHERE
n_volunteers = '1'
AND first_volunteer LIKE '%Первое волонтёрство на 5 вёрст%'
AND run LIKE 'Петергоф Александрийский%'
AND TO_DATE(date, 'DD.MM.YYYY') = (
    SELECT MAX(TO_DATE(date, 'DD.MM.YYYY')) AS date
    FROM volunteers
    )

#### Вступившие в клубы пробегов

In [ ]:
create view view_run_clubs as
WITH ranked_data AS (
    SELECT
        profile_link,
        name,
        date,
        run,  
        n_finishes,
        ROW_NUMBER() OVER (PARTITION BY profile_link ORDER BY TO_DATE(date, 'DD.MM.YYYY') DESC) AS rank
    FROM
        participants
),
last_and_prev_data AS (
    SELECT
        profile_link,
        name,
        MAX(CASE WHEN rank = 1 THEN TO_DATE(date, 'DD.MM.YYYY') END) AS last_date,
        MAX(CASE WHEN rank = 1 THEN n_finishes END) AS last_n_finishes,
        MAX(CASE WHEN rank = 1 THEN run END) AS last_run,
        MAX(CASE WHEN rank = 2 THEN TO_DATE(date, 'DD.MM.YYYY') END) AS prev_date,
        MAX(CASE WHEN rank = 2 THEN n_finishes END) AS prev_n_finishes,
        MAX(CASE WHEN rank = 2 THEN run END) AS prev_run
    FROM
        ranked_data
    WHERE
        rank <= 2
    GROUP BY
        profile_link, name
)
SELECT
    profile_link,
    name,
    last_date,
    last_n_finishes,
    last_run,
    prev_date,
    prev_n_finishes,
    prev_run
FROM
    last_and_prev_data
where last_run like 'Петергоф Александрийский%'
and last_n_finishes in ('10', '25', '50', '100', '150')
and last_date = (select max(TO_DATE(date, 'DD.MM.YYYY')) as ld
from participants
where run like 'Петергоф Александрийский%')

#### Вступившие в клубы волонтёрств

In [ ]:
create view view_help_clubs as
WITH ranked_data AS (
    SELECT
        profile_link,
        name,
        date,
        run,  
        n_volunteers,
        ROW_NUMBER() OVER (PARTITION BY profile_link ORDER BY TO_DATE(date, 'DD.MM.YYYY') DESC) AS rank
    FROM
        volunteers
),
last_and_prev_data AS (
    SELECT
        profile_link,
        name,
        MAX(CASE WHEN rank = 1 THEN TO_DATE(date, 'DD.MM.YYYY') END) AS last_date,
        MAX(CASE WHEN rank = 1 THEN n_volunteers END) AS last_n_volunteers,
        MAX(CASE WHEN rank = 1 THEN run END) AS last_run,
        MAX(CASE WHEN rank = 2 THEN TO_DATE(date, 'DD.MM.YYYY') END) AS prev_date,
        MAX(CASE WHEN rank = 2 THEN n_volunteers END) AS prev_n_volunteers,
        MAX(CASE WHEN rank = 2 THEN run END) AS prev_run
    FROM
        ranked_data
    WHERE
        rank <= 2
    GROUP BY
        profile_link, name
)
SELECT
    profile_link,
    name,
    last_date,
    last_n_volunteers,
    last_run,
    prev_date,
    prev_n_volunteers,
    prev_run
FROM
    last_and_prev_data
where last_run like 'Петергоф Александрийский%'
and last_n_volunteers in ('10', '25', '50', '100', '150')
and last_date = (select max(TO_DATE(date, 'DD.MM.YYYY')) as ld
from participants
where run like 'Петергоф Александрийский%')

#### Вторая суббота в Петергофе

In [ ]:
create view view_second_run as